# P2G 변환

**사용모델** : ```j5ng/et5-typos-corrector```

**선정이유** : 한국어 P2G 모델 중, 대회 규정인 klue-ynat 데이터셋을 사용하여 학습되지 않은 모델을 선택
|
**실험**

- train.csv 파일의 text를 P2G 모델을 이용해 문어체로 변환
- hanspell 맞춤법 교정(띄어쓰기)이 이뤄진 train_spell_checked.csv 파일의 text를 P2G 모델을 이용해 문어체로 변환
- 두 경우의 성능 비교 및 hanspell에서 해결하지 못하던 구어체를 얼마나 해결했는지 확인

### 모델 불러오기 및 테스트

In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
import torch

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [6]:
import sys
print(sys.executable)

/data/ephemeral/.conda/bin/python


In [7]:
model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector',cache_dir='/data/ephemeral/tmp')
tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')

typos_corrector = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    framework="pt",
)

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

In [8]:
input_text = "완죤 어이업ㅅ네진쨬ㅋㅋㅋ"
output_text = typos_corrector("맞춤법을 고쳐주세요: " + input_text,
            max_length=128,
            num_beams=5,
            early_stopping=True)[0]['generated_text']
print(output_text)

완전 어이없네 진짜 ᄏᄏᄏᄏ.


In [22]:
def p2g(sent):
    output = typos_corrector("맞춤법을 고쳐주세요: " + sent,
            max_length=50,
            num_beams=5,
            early_stopping=True)[0]['generated_text']
    return output

In [23]:
g2p_10 = ['당대 최고 포수 양이지 올 시즌 성적 쩌도 신기함니다',
          '이뤄 뢰구긴 관광개 길보니 한구긔 이배…격차 계소 콱때되나',
          '유니테스트 SK하이닉쓰와 칠시보어권 반도체 장비 공급께약',
          '사월부터 통신사 고객쎈터 점심시가넨 긴급쌍담만 반는다',
          '인테리어로 스마트홈 구축…SKT 에코인테리어혀푀와 MOU',
          'SKT 삼심마눤대 스마트폰 갤럭씨 와이드 단독 출시',
          '유엘 리비아 내저 나콰·국쩨저 콱싸 누려…아라변맹 긴그푀의',
          '자고나면 한 명씽 느느 녜비후보…열기 다라오르는 서부산',
          '예상모탄 결과서 노벨상 나온다…게소 견구지워니 중요',
          '제오회 수림문학쌍 오늘부터 공모…장편소설 대상']

In [24]:
g2p_10_result = [p2g(r) for r in g2p_10]

/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [26]:
g2p_10_result

['당대 최고 포수 양이지 올 시즌 성적 쩌도 신기합니다.',
 '이뤄 롯데긴 관광 개길 보니 한구츠 이배 격차 계소 콱 때 되나.',
 '유니테스트 SK하이닉쓰와 칠시 보어권 반도체 장비 공급 약속.',
 '사월부터 통신사 고객 센터 점심 시간이네 긴급 쌍담만 반긴다.',
 '인테리어로 스마트홈 구축 SKT 에코 인테리어 인테리어 해 와 MOU.',
 'SKT 삼심만대 스마트폰 갤럭씨 와이드 단독 출시.',
 '유엘 리비아 내저 나콰 국짜저 콱 싸 누려 hate-speech.',
 '자고 나면 한 명씽 느는 녜비 후보 열기 다라오르는 서부산.',
 '예상 모탄 결과서 노벨상 나온다면서 견구지워니 중요.',
 '제오회 수림 문학 쌍 오늘부터 공모 장편 소설 대상.']

### P2G 교정

In [15]:
import pandas as pd
from tqdm import tqdm

In [13]:
# train 데이터 불러오기
train_df = pd.read_csv('../data/train.csv')
train_df

,ID,text,target,url,date
0,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
1,ynat-v1_train_00001,삼성전자 KBIS 2018서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
2,ynat-v1_train_00002,LG G6 사면 BO 이어폰이 단돈 5천원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00
3,ynat-v1_train_00003,신간 블록체인혁명 2030·남자의 고독사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49
4,ynat-v1_train_00004,이스라엘 정보당국 팔레스타인인 50명 테러 혐의로 체포,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.18. 오후 11:15
...,...,...,...,...,...
6995,ynat-v1_train_06995,힐만 SK 감독 고통스럽지만 내 상황 솔직히 알려야 해,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.10.13 15:46
6996,ynat-v1_train_06996,정의장 사드 국회동의 사안 아니라 쳐도 충분히 협의해야,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.17. 오후 6:20
6997,ynat-v1_train_06997,정치권 엘시티 수사 돌발변수에 촉각…왜 지금,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.16. 오후 6:24
6998,ynat-v1_train_06998,문 대통령 1987 관람…깜짝 방문에 객석 환호·박수종합,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.07. 오후 5:23


In [16]:
# train_df의 text에 apply를 통해 p2g 함수를 적용한 결과를 새로운 컬럼인 p2g_text에 저장
tqdm.pandas()
train_df['p2g_text'] = train_df['text'].progress_apply(p2g)

  0%|          | 10/7000 [00:02<26:10,  4.45it/s]/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 7000/7000 [26:46<00:00,  4.36it/s]


In [17]:
train_df

,ID,text,target,url,date,p2g_text
0,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37,개포2단지 분양 앞두고 개포지구 재건축 불 붙어.
1,ynat-v1_train_00001,삼성전자 KBIS 2018서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33,삼성전자 KBIS 2018서 셰프 컬렉션 선보.
2,ynat-v1_train_00002,LG G6 사면 BO 이어폰이 단돈 5천원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00,LG G6 사면 BO 이어폰이 단돈 5천 원.
3,ynat-v1_train_00003,신간 블록체인혁명 2030·남자의 고독사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49,신간 블록체인 혁명 2030·남자의 고독사.
4,ynat-v1_train_00004,이스라엘 정보당국 팔레스타인인 50명 테러 혐의로 체포,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.18. 오후 11:15,이스라엘 정보당국 팔레스타인인 50명 테러혐의로 체포.
...,...,...,...,...,...,...
6995,ynat-v1_train_06995,힐만 SK 감독 고통스럽지만 내 상황 솔직히 알려야 해,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.10.13 15:46,힐만 SK 감독 고통스럽지만 내 상황 솔직히 알려야 해.
6996,ynat-v1_train_06996,정의장 사드 국회동의 사안 아니라 쳐도 충분히 협의해야,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.17. 오후 6:20,정의장 사드 국회 동의 사항 아니라 쳐도 충분히 협의해야.
6997,ynat-v1_train_06997,정치권 엘시티 수사 돌발변수에 촉각…왜 지금,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.16. 오후 6:24,정치권 엘시티 수사 돌발 변수에 촉각 왜 지금?.
6998,ynat-v1_train_06998,문 대통령 1987 관람…깜짝 방문에 객석 환호·박수종합,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.07. 오후 5:23,문재인 대통령 1988 관람.


In [18]:
# 저장
train_df.to_csv('../data/train_p2g.csv', index=False)

In [19]:
# text와 p2g_text가 같은 경우 1, 다른 경우 0인 컬럼인 is_same을 새로운 컬럼으로 추가

train_df['is_same'] = train_df['text'] == train_df['p2g_text']
train_df

,ID,text,target,url,date,p2g_text,is_same
0,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37,개포2단지 분양 앞두고 개포지구 재건축 불 붙어.,False
1,ynat-v1_train_00001,삼성전자 KBIS 2018서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33,삼성전자 KBIS 2018서 셰프 컬렉션 선보.,False
2,ynat-v1_train_00002,LG G6 사면 BO 이어폰이 단돈 5천원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00,LG G6 사면 BO 이어폰이 단돈 5천 원.,False
3,ynat-v1_train_00003,신간 블록체인혁명 2030·남자의 고독사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49,신간 블록체인 혁명 2030·남자의 고독사.,False
4,ynat-v1_train_00004,이스라엘 정보당국 팔레스타인인 50명 테러 혐의로 체포,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.18. 오후 11:15,이스라엘 정보당국 팔레스타인인 50명 테러혐의로 체포.,False
...,...,...,...,...,...,...,...
6995,ynat-v1_train_06995,힐만 SK 감독 고통스럽지만 내 상황 솔직히 알려야 해,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.10.13 15:46,힐만 SK 감독 고통스럽지만 내 상황 솔직히 알려야 해.,False
6996,ynat-v1_train_06996,정의장 사드 국회동의 사안 아니라 쳐도 충분히 협의해야,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.17. 오후 6:20,정의장 사드 국회 동의 사항 아니라 쳐도 충분히 협의해야.,False
6997,ynat-v1_train_06997,정치권 엘시티 수사 돌발변수에 촉각…왜 지금,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.16. 오후 6:24,정치권 엘시티 수사 돌발 변수에 촉각 왜 지금?.,False
6998,ynat-v1_train_06998,문 대통령 1987 관람…깜짝 방문에 객석 환호·박수종합,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.07. 오후 5:23,문재인 대통령 1988 관람.,False


In [20]:
train_df['is_same'].value_counts()

is_same
False    7000
Name: count, dtype: int64

In [28]:
# p2g_text에서 마지막에 있는 '.'를 제거한 한 결과를 text에 저장

new_train = train_df.copy()
new_train['text'] = new_train['p2g_text'].str[:-1]
new_train.drop(['p2g_text', 'is_same'], axis=1, inplace=True)
new_train

,ID,text,target,url,date
0,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지구 재건축 불 붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
1,ynat-v1_train_00001,삼성전자 KBIS 2018서 셰프 컬렉션 선보,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
2,ynat-v1_train_00002,LG G6 사면 BO 이어폰이 단돈 5천 원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00
3,ynat-v1_train_00003,신간 블록체인 혁명 2030·남자의 고독사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49
4,ynat-v1_train_00004,이스라엘 정보당국 팔레스타인인 50명 테러혐의로 체포,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.18. 오후 11:15
...,...,...,...,...,...
6995,ynat-v1_train_06995,힐만 SK 감독 고통스럽지만 내 상황 솔직히 알려야 해,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.10.13 15:46
6996,ynat-v1_train_06996,정의장 사드 국회 동의 사항 아니라 쳐도 충분히 협의해야,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.17. 오후 6:20
6997,ynat-v1_train_06997,정치권 엘시티 수사 돌발 변수에 촉각 왜 지금?,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.16. 오후 6:24
6998,ynat-v1_train_06998,문재인 대통령 1988 관람,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.07. 오후 5:23


In [29]:
# 저장
new_train.to_csv('../data/train_p2g.csv', index=False)